In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly.io as pio

from htc.utils.helper_functions import sort_labels
from htc.utils.visualization import create_median_spectra_comparison_figure
from htc_projects.rat.settings_rat import settings_rat
from htc_projects.species.settings_species import settings_species
from htc_projects.species.tables import baseline_table

pio.kaleido.scope.mathjax = None

In [2]:
df = baseline_table(settings_species.label_mapping_organs)

# Median spectra per subject
df = df.groupby(["subject_name", "species_name", "label_name"], as_index=False).agg(
    median_normalized_spectrum=pd.NamedAgg(
        column="median_normalized_spectrum", aggfunc=lambda c: np.mean(np.stack(c), axis=0)
    ),
)

df = sort_labels(df, sorting_cols=["label_name", "species_name"])
df.head()

,subject_name,species_name,label_name,median_normalized_spectrum
0,SPACE_000001,human,stomach,"[0.0044934, 0.0044833585, 0.004338775, 0.00414..."
1,SPACE_000002,human,stomach,"[0.003292484, 0.003347599, 0.0032964824, 0.003..."
2,SPACE_000004,human,stomach,"[0.004564302, 0.0045444965, 0.0044127274, 0.00..."
3,SPACE_000006,human,stomach,"[0.0032123737, 0.0032752852, 0.0033065875, 0.0..."
4,SPACE_000008,human,stomach,"[0.0024084495, 0.0024103245, 0.0023038439, 0.0..."


In [3]:
fig = create_median_spectra_comparison_figure(
    df,
    group_column="species_name",
    color_mapping=settings_species.species_colors,
    n_cols=4,
)
fig.update_layout(font_family="Libertinus Sans", font_size=16)
fig.update_layout(showlegend=False)
fig.update_layout(width=1200)
fig.update_annotations(font=dict(size=20))
for i, annotation in enumerate(fig.layout.annotations):
    fig.layout.annotations[i].text = settings_rat.labels_paper_renaming.get(
        fig.layout.annotations[i].text, fig.layout.annotations[i].text
    )

fig.update_layout(title=None)
fig.write_image(settings_species.paper_dir / "median_spectra_species_base.pdf")
fig